In [1]:
data_var = '2024-01-28'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13735,2024-01-28,Japão B.League,03:05,Diamond Dolphins,Sendai,1.40,2.80,159.5,1.87,1.79,-6.5,1.95,1.76,UuY7d3cL,0.714286,0.357143,0.534759,0.558659,0.071429,178.394,43.152698,0.241895,1.2,1.643168,1.369306,235.00,2.270,0.171318,0.075471,-27.0,251.896,155.769919,0.618390,1.8,1.643168,0.912871,176.00,3.296,2.190703,0.664655,-15.0,94,80,2.50,2.20,150.304,290.394,0.471405,0.030912,0.072426,2.400000e-01,4.800000e-02,8.333333e+00,0.612092,0.7,0.087908,0.92,0.184,9.782609,0.389943,0.5,0.110057
13736,2024-01-28,Japão B.League,03:05,Alvark,Osaka,1.04,9.45,155.5,1.85,1.85,-18.5,2.10,1.66,67nDzMz2,0.961538,0.105820,0.540541,0.540541,0.067359,104.128,14.013312,0.134578,3.0,0.000000,0.000000,88.74,1.182,0.122556,0.103685,104.0,251.390,128.509921,0.511197,0.6,1.341641,2.236068,150.15,2.994,1.590387,0.531191,-39.0,87,91,1.02,1.65,96.080,312.730,1.133798,0.000000,0.165493,-3.300000e-01,-6.600000e-02,-6.060606e-01,0.870573,0.7,-0.170573,-5.00,-1.000,-8.450000,0.385639,0.1,-0.285639
13737,2024-01-28,Japão B.League,02:05,Yokohama,Shimane,3.20,1.32,164.5,1.85,1.85,5.5,2.08,1.67,rPVea5D2,0.312500,0.757576,0.540541,0.540541,0.070076,167.048,43.126240,0.258167,1.8,1.643168,0.912871,208.55,1.942,0.649746,0.334576,-7.0,100.514,10.469216,0.104157,1.8,1.643168,0.912871,86.70,1.210,0.187083,0.154614,25.0,97,85,2.15,1.02,208.792,135.210,0.588213,0.000000,0.154621,5.760000e+00,1.152000e+00,1.909722e+00,0.432969,0.4,-0.032969,-1.44,-0.288,-1.111111,0.602348,0.3,-0.302348
13738,2024-01-28,Japão B.League,01:05,Toyama,Chiba,5.03,1.15,169.5,1.85,1.85,10.5,1.97,1.74,MFUabPS8,0.198807,0.869565,0.540541,0.540541,0.068372,406.904,293.410510,0.721080,0.0,0.000000,NaN,250.74,5.168,3.471501,0.671730,-44.0,137.188,40.011267,0.291653,2.4,1.341641,0.559017,102.35,1.416,0.424005,0.299438,72.0,63,89,3.98,1.15,410.704,130.852,0.887888,0.000000,0.087674,-1.140000e+00,-2.280000e-01,-1.767544e+01,0.245798,0.1,-0.145798,-2.26,-0.452,-0.331858,0.641999,0.6,-0.041999
13739,2024-01-28,Japão B.League,02:05,Hokkaido,Shinshu,1.33,3.12,153.5,1.81,1.85,-8.5,2.04,1.69,Gx8Ys4rS,0.751880,0.320513,0.552486,0.540541,0.072393,271.810,148.254644,0.545435,1.2,1.643168,1.369306,412.46,3.406,1.900087,0.557865,-2.0,421.450,162.060023,0.384530,0.6,1.341641,2.236068,632.70,6.814,3.111749,0.456670,-70.0,82,57,5.03,11.10,145.782,464.566,0.568863,0.015456,0.132701,-2.220446e-16,-4.440892e-17,-7.430939e+15,0.000000,0.0,0.000000,0.98,0.196,10.816327,0.198214,0.1,-0.098214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13926,2024-01-28,Eua Ncaa,15:00,La Lafayette,Texas State,1.30,3.47,143.5,1.83,1.83,-8.5,1.91,1.80,b3nX6sN7,0.769231,0.288184,0.546448,0.546448,0.057415,155.502,31.239258,0.200893,1.8,1.643168,0.912871,168.08,2.036,0.608136,0.298692,22.0,194.834,98.164651,0.503837,0.6,1.341641,2.236068,244.40,2.706,1.179292,0.435806,-28.0,88,65,1.91,3.76,145.890,233.400,0.643363,0.000000,0.041931,1.800000e+00,3.600000e-01,8.333333e-01,0.000000,0.0,0.000000,-3.35,-0.670,-3.686567,0.320080,0.4,0.079920
13927,2024-01-28,Eua Ncaa,15:00,Rutgers,Purdue,4.89,1.18,141.5,1.80,1.91,10.5,1.83,1.83,Aw68XWCm,0.2

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13756,13:00,Itália Série A1 Feminina,San Giovanni F,Ragusa F,1.65,Back Home
13809,13:30,Polônia Liga De Basquete,Lublin,Dabrowa Gornicza,1.80,Back Home
13866,20:00,Eua Nba,Orlando Magic,Phoenix Suns,2.12,Back Home
13895,07:30,Filipinas Copa Commissioners,Ginebra Kings,San Miguel Beermen,1.64,Back Home
13896,11:30,Finlândia I Divisão A,Helsingin NMKY,Huima,1.82,Back Home
